=================================================

MILESTONE 3 PHASE 2

- Nama  : Bimantyo Arya Majid

- Objective : Program ini dibuat untuk memeriksa apakah data mematuhi seperangkat aturan, kriteria, atau standar tertentu. Tujuan utamanya adalah untuk memverifikasi bahwa data tersebut benar secara struktural dan sintaksis serta mengikuti format atau pola yang telah ditentukan sebelumnya.

=================================================

# 1. Melakukan Proses Install Great Expectation Package

Proses install GE akan dilakukan menggunakan versi 0.18.19 karena untuk versi terbaru akan berpotensi error

In [1]:
# Install the library

!pip install -q "great-expectations==0.18.19"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 42.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


# 2. Mendefinisikan Data Context

Data context merupakan modul utama untuk melakukan data validation, dengan adanya data context bisa melakukan End to End Validation. Di dalam data context terbagi menjadi 4 sub,
- yang pertama mengenai setup. Mengkoneksikan GE dengan asal sourcenya
- yang ke-dua mengenai connecting data. Setelah setup selesai baru connect data menggunakan data .csv
- yang ke-tiga mengenai expectationnya
- yang ke-empat mengenai rulesnya

In [1]:
# Membuat Data Context
from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='./')

# 3. Menghubungkan dengan Data Source

Menghubungkan Data Context dengan sumber data yang digunakan. Data yang digunakan merupakan data clean yang sudah di dapatkan dari Pipeline di airflow

In [2]:
# 1. Mendefinisikan template sumber data
datasource_name = 'milestone3'
datasource = context.sources.add_pandas(datasource_name)

# 2. Tambah asset CSV dari file lokal
asset = datasource.add_csv_asset(
    name="restaurant_data",
    filepath_or_buffer="/content/P2M3_Bimantyo_arya_data_clean.csv"
)

# 3. Build batch request
batch_request = asset.build_batch_request()

# 4. Membuat Rules untuk Great Expectation

Expectation suite adalah sekumpulan aturan atau validasi (expectations) yang ingin diterapkan terhadap data yang ingin digunakan. Expectation suite digunakan untuk memastikan data yang digunakan sesuai dengan standar kualitas yang diharapkan.

In [3]:
# Membuat nama suite
expectation_suite_name = 'expectation-restaurant-dataset'  # Nama suite

# Membuat suite (kalau belum ada), atau update (kalau sudah ada)
context.add_or_update_expectation_suite(expectation_suite_name)

# Membuat validator (untuk membuat atau mengecek expectations terhadap batch data)
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Mengecek isi data (opsional untuk lihat 5 baris pertama)
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,name,location,cuisine,rating,seating_capacity,average_meal_price,marketing_budget,social_media_followers,chef_experience_years,number_of_reviews,avg_review_length,ambience_score,service_quality_score,parking_availability,weekend_reservations,weekday_reservations,revenue
0,Restaurant 0,Rural,Japanese,4.0,38,73.98,2224,23406,13,185,161.924906,1.3,7.0,True,13,4,638945.52
1,Restaurant 1,Downtown,Mexican,3.2,76,28.11,4416,42741,8,533,148.759717,2.6,3.4,True,48,6,490207.83
2,Restaurant 2,Rural,Italian,4.7,48,48.29,2796,37285,18,853,56.849189,5.3,6.7,False,27,14,541368.62
3,Restaurant 3,Rural,Italian,4.4,34,51.55,1167,15214,13,82,205.433265,4.6,2.8,True,9,17,404556.80
4,Restaurant 4,Downtown,Japanese,4.9,88,75.98,3639,40171,9,78,241.681584,8.6,2.1,False,37,26,1491046.35


## 4.1 Expectations

Expectations adalah pernyataan yang dapat diverifikasi yang berkaitan dengan dataset yang akan di uji validasi. Uji validasi ekspektasi akan dilakukan sebanyak 7 kali yaitu :

1. to be unique
2. to be between min_value and max_value
3. to be in set
4. to be in type list
5. to be in median between min_value and max_value
6. to be in unique value count between min_value and max_value
7. to be in minimum between min_value and max_value

### 4.1.1 EXPECTATION 1

In [8]:
# Expectation 1 kolom `name` memiliki data yang unik setiap barisnya

validator.expect_column_values_to_be_unique('name')

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_unique",
    "kwargs": {
      "column": "name",
      "batch_id": "milestone3-restaurant_data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 8368,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

- JUSTIFIKASI EXPECTATION 1

Expectation 1 menggunakan to be unique dan diterapkan terhadap kolom ```name``` karena untuk memastikan bahwa di dalam dataset, setiap data restaurant tidak ada yang bersifat duplikat, karena jika ditemukan data yang duplikat maka dapat dikatakan bahwa data restoran tersebut bisa dianggap tidak valid. Kolom name sering digunakan untuk mengidentifikasi sebuah entitas. Dataset ini memiliki lebih dari 8000 data restaurant dan setiap restaurant mewakili nama mereka sendiri.

### 4.1.2 EXPECTATION 2

In [9]:
# Expectation 2 kolom `rating` memiliki rentang skala dari 0-5

validator.expect_column_values_to_be_between(
    column="rating",
    min_value=0,
    max_value=5
)

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "column": "rating",
      "min_value": 0,
      "max_value": 5,
      "batch_id": "milestone3-restaurant_data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 8368,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

- JUSTIFIKASI EXPECTATION 2

Expectation 2 menggunakan to be between min_value and max_value dan diterapkan terhadap kolom ```rating``` karena untuk memastikan bahwa di dalam dataset, setiap data restaurant memiliki standar rating yang sama. Standar review rating setiap resturant oleh customer umumnya menggunakan skaa dari 0-5 seperti di Google, Tokopedia dan Gojek untuk menilai kualitas layanan/restoran. Jika nilai di luar rentang tersebut kemungkinan bisa terjadi kesalahan saat input, data error, format skala rating yang digunakan berbeda. Selain itu dengan memastikan skala rating dari 0-5 juga memastikan untuk menghindari terjadinya bias jika ada salah satu restoran yang memiliki review > 5 atau < 0

### 4.1.3 EXPECTATION 3

In [7]:
# Expectation 3 kolom `cuisine` memiliki nilai yang sudah ditentukan

validator.expect_column_values_to_be_in_set(
    column="cuisine",
    value_set=["American", "French", "Indian", "Italian", "Japanese", "Mexican"]
)

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_in_set",
    "kwargs": {
      "column": "cuisine",
      "value_set": [
        "American",
        "French",
        "Indian",
        "Italian",
        "Japanese",
        "Mexican"
      ],
      "batch_id": "milestone3-restaurant_data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 8368,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

- JUSTIFIKASI EXPECTATION 3

Expectation 3 menggunakan to be in set dan diterapkan terhadap kolom ```cuisine``` karena untuk memastikan bahwa di dalam dataset, jenis masakan yang ingin di analisa meliputi 6 kategori saja yaitu ```American, French, Indian, Italian, Japanese, Mexican ``` jika ada jenis masakan diluar 6 kategori tersebut dapat dikatakan data tidak valid. Validasi ini juga digunakan untuk menghindari Error apabila terdapat Data input yang salah, seperti lowercase atau miss type tanda baca.

### 4.1.4 EXPECTATION 4

In [10]:
# Expectation 4 kolom `revenue` memiliki tipe data integer atau float

validator.expect_column_values_to_be_in_type_list(
    column="revenue",
    type_list=["int", "float"]
)

  warnings.warn(



Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_in_type_list",
    "kwargs": {
      "column": "revenue",
      "type_list": [
        "int",
        "float"
      ],
      "batch_id": "milestone3-restaurant_data"
    },
    "meta": {}
  },
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

- JUSTIFIKASI EXPECTATION 4

Expectation 4 menggunakan to be in type list dan diterapkan terhadap kolom ```revenue``` karena untuk memastikan bahwa di dalam dataset, tipe data yang terdapat pada kolom revenue merupakan integer atau float. Karena revenue merupakan total pendapatan restaurant, dimana total pendapatan akan menghasilkan angka dan angka tersebut harus memiliki tipe data float atau integer. Jika tipe data yang dimiliki kolom revenue merupakan "String" maka nantinya data tidak akan bisa dibaca di dalam visualisasi kibana.

### 4.1.5 EXPECTATION 5

In [12]:
# Expectation 5 kolom `ambience_score` memiliki skala 0-10

validator.expect_column_median_to_be_between(
    column="ambience_score",
    min_value=0,
    max_value=10
)

  warnings.warn(



Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_median_to_be_between",
    "kwargs": {
      "column": "ambience_score",
      "min_value": 0,
      "max_value": 10,
      "batch_id": "milestone3-restaurant_data"
    },
    "meta": {}
  },
  "result": {
    "observed_value": 5.5
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

- JUSTIFIKASI EXPECTATION 5

Expectation 5 menggunakan to be in median between min_valu and max_value dan diterapkan terhadap kolom ```ambience_score``` karena nilai ambience_score adalah nilai yang bersifat subjektif dan rawan akan outlier, penilaian ambience merupakan sesuatu yang subjektif dan setiap orang bisa memiliki presepsi berbeda. Ada kemungkinan bahwa customer akan memberikan skor yang sangat rendah atau bahkan skor yang sangat tinggi. Nilai median juga mencerminkan presepsi mayoritas customer.

### 4.1.6 EXPECTATION 6

In [13]:
# Expectation 6 kolom `parking_availability` memastikan hanya berisi 2 nilai saja
validator.expect_column_unique_value_count_to_be_between(
    column="parking_availability",
    min_value=2,
    max_value=2
)

  warnings.warn(



Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_unique_value_count_to_be_between",
    "kwargs": {
      "column": "parking_availability",
      "min_value": 2,
      "max_value": 2,
      "batch_id": "milestone3-restaurant_data"
    },
    "meta": {}
  },
  "result": {
    "observed_value": 2
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

- JUSTIFIKASI EXPECTATION 6

Expectation 6 menggunakan to be in unique value count between min_value and max_valuedan diterapkan terhadap kolom ```parking_availability``` karena untuk validasi konsistensi nilai yang bersifat boolean. Untuk kolom parking_availability hanya memiliki 2 kemungkinan Tersedia atau tidak tersedia, dimana sudah dilakukan convert menjadi TRUE dan FALSE untuk memudahkan dalam visualisasi di kibana nantinya. Jika expectation menghasilkan false maka terdapat lebih dari 2 nilai atau bahkan hanya memiliki 1 nilai saja bisa jadi true/ false saja atau NaN.

### 4.1.7 EXPECTATION 7

In [14]:
# Expectation 7 kolom `marketing_budget` memastikan bahwa setiap restaurant memiliki marketing budget
validator.expect_column_min_to_be_between(
    column="marketing_budget",
    min_value=0,
    max_value=10000
)

  warnings.warn(



Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_min_to_be_between",
    "kwargs": {
      "column": "marketing_budget",
      "min_value": 0,
      "max_value": 10000,
      "batch_id": "milestone3-restaurant_data"
    },
    "meta": {}
  },
  "result": {
    "observed_value": 604
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

- JUSTIFIKASI EXPECTATION 7

Expectation 7 menggunakan to be in unique value count between min_value and max_valuedan diterapkan terhadap kolom ```marketing_budget``` karena untuk memastikan bahwa setiap restaurant memiliki marketing budget. Jika ditemukan nilai 0 di dalam kolom marketing_budget atau nilai NaN dapat menandakan bahwa terjadi error input, karena setiap usaha bisnis pasti memiliki budget untuk memasarkan produk/ jasa mereka jadi tidak mungkin bahwa sebuah bisnis tidak menyiapkan dana untuk pemasaran.

# 5. Menyimpan Expectation Suite

In [15]:
# Save into Expectation Suite

validator.save_expectation_suite()

# 6. Checkpoint

Berfungsi untuk membandingkan rules yang sudah dibuat dengan data

In [16]:
# Membuat checkpoint

checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

In [17]:
# Menjalankan checkpoint

checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/27 [00:00<?, ?it/s]